# Sequential model

## [Getting started with the Keras Sequential model](https://keras.io/getting-started/sequential-model-guide/#getting-started-with-the-keras-sequential-model)

The [Sequential Model](https://keras.io/getting-started/sequential-model-guide/) is a linear stak of layers.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

Using TensorFlow backend.


In [62]:
model = Sequential([
    Dense(32, input_shape=(784,)), # first hidden layer, the first parameter specifies the number of neurons
    Activation('relu'),
    Dense(10), # the final layer will be the output layer, therefore there are 10 output neurons
    Activation('softmax'),
])

[Dense](https://keras.io/layers/core/#dense) implements the operation: `output = activation(dot(input, kernel) + bias)` where `activation` is the element-wise activation function passed as the `activation` argument, `kernel` is a weights matrix created by the layer, and `bias` is a bias vector created by the layer (only applicable if `use_bias` is `True`). Default activation is the identity function. We also have the option to initialize and regularize the kernel and bias. Refer the [initializers](initializers_regularizers.ipynb#Initializers) and [regularizers](initializers_regularizers.ipynb#Regularizers) notebooks respectively.
 
*Note:* If the input to the layer has a rank greater than 2, then it is flattened prior to the initial dot product with `kernel`.

All the possible layers in keras can be [found here](https://keras.io/layers/core/).

You can also simply add layers via the `.add()` method:

In [2]:
# as first layer in a sequential model:
model = Sequential()
model.add(Dense(32, input_dim=784)) # input_dim is another way to speify the size of input
model.add(Activation('relu'))
# now the model will take as input arrays of shape (*, 16)
# and output arrays of shape (*, 32)

# after the first layer, you don't need to specify
# the size of the input anymore:
model.add(Dense(10, activation='softmax'))

### [Activations](https://keras.io/activations/)

Activations can either be used through an Activation layer, or through the activation argument supported by all forward layers. Check out the documentation for the different kinds of activation functions possible.

### Input and Output shapes
We only need to specify the input shape for the first layer, because the following layers can do automatic shape inference. In `input_shape`, the batch dimension is not included.

If you ever need to specify a fixed batch size for your inputs (this is useful for stateful recurrent networks), you can pass a `batch_size` argument to a layer. The most common situation would be a 2D input with shape (batch_size, input_dim). For a 2D input with shape (batch_size, input_dim), the output would have shape (batch_size, units).

## [Compilation](https://keras.io/getting-started/sequential-model-guide/#compilation)

Before training a model, you need to configure the learning process, which is done via the compile method. It receives three arguments:


- [optimizer](https://keras.io/optimizers/):  This could be the string identifier of an existing optimizer (such as `rmsprop` or `adagrad`), or an instance of the Optimizer class.
```
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)
```
or
```
### pass optimizer by name: default parameters will be used
model.compile(loss='mean_squared_error', optimizer='sgd')
```
- [loss function](https://keras.io/losses/):  This is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as `categorical_crossentropy` or `mse`), or it can be an objective function.
- [list of metrics](https://keras.io/metrics/):  For any classification problem you will want to set this to `metrics=['accuracy']`. A metric could be the string identifier of an existing metric or a custom metric function. A metric function is similar to a loss function, except that the results from evaluating a metric are not used when training the model.


In [3]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

## [Training](https://keras.io/getting-started/sequential-model-guide/#training)

Keras models are trained on Numpy arrays of input data and labels. For training a model, you will typically use the `fit` function.

### Example 1: Two-class classification

In [2]:
# For a single-input model with 2 classes (binary classification):

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 0.7095 - acc: 0.4880     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 0.7017 - acc: 0.5050     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 0.6956 - acc: 0.5160     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.6927 - acc: 0.5310     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.6911 - acc: 0.5220     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.6870 - acc: 0.5450     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.6843 - acc: 0.5430     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.6802 - acc: 0.5590     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 0.6775 - acc: 0.5800     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 0.6754 - acc: 0.5980     


#### Accuracy of this model on training data

In [3]:
pred = model.predict(data)
from sklearn.metrics import confusion_matrix
cmat = confusion_matrix(pred.round(2)>0.5, labels)
print("accuracy on training data =", cmat.diagonal().sum()/cmat.sum())
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

accuracy on training data = 0.577


#### Improving accuracy with more epochs

We can improve the accuracy on the training set drastically by iterating for more epochs. But this is not how the number of epochs should be chosen in pratice because this might lead to overfitting.

Set `verbose=0` to avoid printing all the interations.

In [4]:
model.fit(data, labels, epochs=50, batch_size=32, verbose=0)
pred = model.predict(data)
from sklearn.metrics import confusion_matrix
cmat = confusion_matrix(pred.round(2)>0.5, labels)
print("accuracy on training data =", cmat.diagonal().sum()/cmat.sum())

accuracy on training data = 0.834


#### Visualizing the model
```
from keras.utils import plot_model
plot_model(model, to_file='bin_classification.png')
```

### Example 2: Multi-class classification

In [ ]:
from keras.utils import to_categorical

# For a single-input model with 10 classes (categorical classification):
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=150, batch_size=32, verbose = 0)

# Accuracy on the training data
pred = model.predict(data)

# taking the maximum output as the predicted class label and then building the confusion matrix
cmat = confusion_matrix(pred.argmax(axis = 1), labels)
print(cmat)
print("accuracy on training data =", cmat.diagonal().sum()/cmat.sum())

In [53]:
# For a single-input model with 10 classes (categorical classification):
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to 4 digit binary encoding
y = []
for x in labels:
    temp = [int(i) for i in list(bin(x[0]).split('b')[1])]
    pretemp = [0 for i in range(4-len(temp))]
    y.append(pretemp+temp)
    
bin_encoding = np.array(y)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, bin_encoding, epochs=150, batch_size=32, verbose = 0)

# Accuracy on the training data
pred = model.predict(data)
pred>0.5
# taking the maximum output as the predicted class label and then building the confusion matrix
# cmat = confusion_matrix([int(''.join([str(i) for i in arr]), 2) for arr in pred], labels)
# print(cmat)
# print("accuracy on training data =", cmat.diagonal().sum()/cmat.sum())



array([[False, False, False,  True],
       [False, False, False,  True],
       [False, False, False, False],
       ..., 
       [False, False,  True, False],
       [False, False,  True, False],
       [False, False, False, False]], dtype=bool)

In [51]:
np.random.seed(0)
labels = np.random.randint(10, size=(10, 1))
y = []
for x in labels:
    temp = [int(i) for i in list(bin(x[0]).split('b')[1])]
    pretemp = [0 for i in range(4-len(temp))]
    y.append(pretemp+temp)
    
np.array(y)

[int(''.join([str(i) for i in arr]), 2) for arr in y]

[5, 0, 3, 3, 7, 9, 3, 5, 2, 4]

### Example 3: Multilayer Perceptron (MLP) for multi-class softmax classification:

In [64]:
# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 0s - loss: 2.3678 - acc: 0.1120     
Epoch 2/20
1000/1000 [==============================] - 0s - loss: 2.3730 - acc: 0.0880     
Epoch 3/20
1000/1000 [==============================] - 0s - loss: 2.3391 - acc: 0.0860     
Epoch 4/20
1000/1000 [==============================] - 0s - loss: 2.3086 - acc: 0.1130     
Epoch 5/20
1000/1000 [==============================] - 0s - loss: 2.3175 - acc: 0.1020     
Epoch 6/20
1000/1000 [==============================] - 0s - loss: 2.3225 - acc: 0.0920     
Epoch 7/20
1000/1000 [==============================] - 0s - loss: 2.3101 - acc: 0.1040     
Epoch 8/20
1000/1000 [==============================] - 0s - loss: 2.3117 - acc: 0.1110     
Epoch 9/20
1000/1000 [==============================] - 0s - loss: 2.3147 - acc: 0.1080     
Epoch 10/20
1000/1000 [==============================] - 0s - loss: 2.3052 - acc: 0.1060     
Epoch 11/20
1000/1000 [==============================] - 0s - loss: 2

[2.2967255115509033, 0.14000000059604645]